In [ ]:
def rasterization(cam: camera, canvas: list, height_map: list, verticies: list, faces: list, border_color: tuple, fill_color: tuple):
    for face in faces:
        ax, ay, az, _ = verticies[face[0]]
        bx, by, bz, _  = verticies[face[1]]
        cx, cy, cz, _  = verticies[face[2]]
        vector1 = np.array([bx-ax, by-ay, bz-az])
        vector2 = np.array([bx-cx, by-cy, bz-cz])
        bearing = np.cross(vector1, vector2)

        #######FIX DIVIDING BY ZERO ERROR########### still might have some potential errors here
        if bearing[2] == 0:
            x_slope = 999999999999999999999999999999
            y_slope = 999999999999999999999999999999
        else:
            if bearing[0] == 0:
                x_slope = 0
            else:
                x_slope = (-1)/(bearing[2]/bearing[0])
            if bearing[1] == 0:
                y_slope = 0
            else:
                y_slope = (-1)/(bearing[2]/bearing[1])


        z_intercept = verticies[face[0]][2] - verticies[face[0]][0]*x_slope - verticies[face[0]][1]*y_slope
        overlap_counter = 0
        shape = np.zeros([cam.resolution[0], cam.resolution[1]]) 
        for idx, point_idx in enumerate(face):
            for x,y in line_2d([round(verticies[point_idx][0]),round(verticies[point_idx][1])],[round(verticies[face[idx-1]][0]),round(verticies[face[idx-1]][1])]):
                if 0 <= x <= cam.resolution[0]-1 and 0 <= y <= cam.resolution[1]-1:
                    if shape[x][y] == 1:
                        overlap_counter += 1
                    else:
                        shape[x][y] = 1 
                    z = z_intercept + x*x_slope + y*y_slope
                    if height_map[x][y] <= z <= cam.view_distance[0]:
                        canvas[x][y] = border_color
                        height_map[x][y] = z
        if overlap_counter < sum(sum(shape))/4:  
            mid = sum(verticies[point_idx][:2] for point_idx in face)/len(face)
            middle_point = np.append(mid, z_intercept+mid[0]*x_slope+mid[1]*y_slope)
            if 0 <= middle_point[0] <= cam.resolution[0]-1 or 0 <= middle_point[1] <= cam.resolution[1]-1: 
                valid_points = [verticies[point_idx] for point_idx in face if 0 <= verticies[point_idx][0] <= cam.resolution[0]-1 and 0 <= verticies[point_idx][1] <= cam.resolution[1]-1]
                if valid_points:
                    inside_point = sum(valid_points)/len(valid_points)
                    for x,y in line_2d((round(middle_point[0]),round(middle_point[1])),(round(inside_point[0]),round(inside_point[1]))):
                        if 0 <= x <= cam.resolution[0]-1 and 0 <= y <= cam.resolution[1]-1:
                            middle_point = np.asarray([x,y,z_intercept+x*x_slope+y*y_slope])
                            break
                else:
                    continue # loops back to the next face because this face is completely off screen

            stack = [middle_point]
        
            counter = 0
            old_shape = shape.copy()

            while stack:
                x,y,z = stack[0]
                x,y = round(x), round(y)
                stack.pop(0)
                if 0 <= x <= cam.resolution[0]-1 and 0 <= y <= cam.resolution[1]-1: 
                    if not shape[x][y]:
                        if z >= height_map[x][y]: 

                            counter += 1

                            canvas[x][y] = fill_color
                            height_map[x][y] = z
                        stack.append(np.asarray([x+1,y,z_intercept+(x+1)*x_slope+y*y_slope]))
                        stack.append(np.asarray([x-1,y,z_intercept+(x-1)*x_slope+y*y_slope]))
                        stack.append(np.asarray([x,y+1,z_intercept+x*x_slope+(y+1)*y_slope]))
                        stack.append(np.asarray([x,y-1,z_intercept+x*x_slope+(y-1)*y_slope]))
                    shape[x][y] = 1 

            counter2 = 0
            if counter >= 200000:
                print(bearing)
                visualize_shape = initialize_2dimage((1000,1000),(255,255,255))
                for idx, line in enumerate(old_shape):
                    for idx2, point in enumerate(line):
                        if point == 1:
                            counter2 += 1
                            print(counter2)
                            visualize_shape[idx][idx2] = (0,0,0)
                        stack = [middle_point]
                if visualize_shape[round(middle_point[0])][round(middle_point[1])] == (0,0,0):
                    visualize_shape[round(middle_point[0])][round(middle_point[1])] = (200,0,0) #is part of shape
                else:
                    visualize_shape[round(middle_point[0])][round(middle_point[1])] = (0,0,200) #isn't part of shape
                p = list(zip(*visualize_shape))[::-1]
                a = np.array(p, dtype=np.uint8)
                new_image = Image.fromarray(a)
                display(new_image)
            
    return canvas